In [ ]:
import sys
from time import sleep
sys.path.append('..')
from pprint import pprint
import time
from RaAuto.Pmo import OpenPMO, HandleInternalMail
from RaAuto.RaAuto import *
from RaAuto.RaTime import add_minutes
import pyautogui as pag
from PIL import Image, ImageDraw
import pyperclip
from RaAuto.RaImage import capture_and_ocr, imshow
import pandas as pd
from datetime import datetime
import json
%matplotlib inline

# Date functions

In [ ]:

def string_to_date(date):
    """
    Converts a date string in the format 'YYYY-MM-DD' to a datetime object.

    Args:
        date (str): Date string in the format 'YYYY-MM-DD'.

    Returns:
        datetime: A datetime object representing the date.
    """
    if isinstance(date, str):
        return datetime.strptime(date, "%Y-%m-%d")
    return date

def date_to_string(date):
    """
    Converts a datetime object to a string in the format 'YYYY-MM-DD'.

    Args:
        date (datetime): A datetime object.

    Returns:
        str: A string representation of the date in the format 'YYYY-MM-DD'.
    """

    if isinstance(date, str):     
        return date.strftime("%Y-%m-%d")
    return date

def date_in_range(date, date_start, date_end):
    """
    Checks if a date is within a specified range.

    Args:
        date (str or datetime): The date to check.
        date_start (str or datetime): The start of the range.
        date_end (str or datetime): The end of the range.

    Returns:
        bool: True if the date is within the range, False otherwise.
    """
    # Convert inputs to datetime if they are strings
    if isinstance(date, str):
        date = string_to_date(date)
    if isinstance(date_start, str):
        date_start = string_to_date(date_start)
    if isinstance(date_end, str):
        date_end = string_to_date(date_end)
    
    return date_start <= date <= date_end

In [ ]:
class ImgsEnum:
    def __init__(self, imagesPath):
        self.Orsak = os.path.join(imagesPath, "Waitinglist", "Orsak.png")
        self.Kategori = os.path.join(imagesPath, "Waitinglist", "Kategori.png")
        self.Tidsatgang = os.path.join(imagesPath, "Waitinglist", "Tidsatgang.png")
        self.Patienter = os.path.join(imagesPath, "Waitinglist", "Patienter.png")
        self.BokaSenast = os.path.join(imagesPath, "Waitinglist", "BokaSenast.png")
        self.WaitingListMenuBar = os.path.join(imagesPath, "Waitinglist", "WaitingListMenuBar.png")
        self.Avbryt = os.path.join(imagesPath, "Waitinglist", "Avbryt.png")
        self.Avbryt2 = os.path.join(imagesPath, "Waitinglist", "Avbryt2.png")
        self.Varning = os.path.join(imagesPath, "Waitinglist", "Varning.png")
        self.ArAvliden = os.path.join(imagesPath, "Waitinglist", "ArAvliden.png")
        self.Radera = os.path.join(imagesPath, "Waitinglist", "Radera.png")

IMG = ImgsEnum(os.path.join("..", "Images"))

In [ ]:
def ScrollUp():
    sleep(0.2)
    pag.scroll(500)
    sleep(0.2)
    pag.scroll(500)
    sleep(0.2)

# Read VL from PMO

In [ ]:
file_path = "..//..//Files//"
current_personnr_limit_first_part = "870101" # Sista personen i "redo att kallas nu"
current_personnr_limit_last_part = "1234"
final_personnr_limit_first_part = "870101" # Sista personen i hela listan
final_personnr_limit_last_part = "1234"

In [ ]:
def check_alive():
    pos = waitForElement(IMG.Varning, limit=1, warn=False)
    if pos: ## Patienten avliden
        waitAndPress("enter")
        sleep(2)
        return False

    return True

In [ ]:
result = []
OpenPMO()
for i in range(500):
    waitAndPress("alt", "A")
    waitForElement(IMG.Avbryt)
    sleep(0.5)

    alive = check_alive()

    pos = waitAndClickElement(IMG.Orsak, relX=100, relY=100, beforeStart=0.5)
    if pos:
        waitAndPress("ctrl", "A")
        waitAndPress("ctrl", "C")
        sleep(0.4)
        orsak = str(pyperclip.paste())
    else:
        break

    if not "ADM " in orsak.upper():        
        
        pos = waitAndClickElement(IMG.Kategori, relX=60)
        if pos:
            waitAndType("lä")
            waitAndPress("enter")
        else:
            break


        pos = waitAndClickElement(IMG.Tidsatgang, relX=60)
        if pos:
            waitAndPress("ctrl", "A")
            waitAndPress("delete")
            waitAndType("30")
            waitAndPress("enter")
        else:
            break

            

        pos = waitForElement(IMG.Patienter)
        if pos:
            personnr = capture_and_ocr(pos.x-14, pos.y+12, 100, 20)
            personnr = personnr.replace("—", "-")
        else:
            break

        pos = waitAndClickElement(IMG.BokaSenast, relY=15)
        if pos:
            waitAndPress("ctrl", "A")
            waitAndPress("ctrl", "C")
            sleep(0.4)
            date = str(pyperclip.paste())
        else:
            break

        if not alive:
            print(f"{date}, {personnr} är avliden")
            pos = waitAndClickElement(IMG.Avbryt)
            if pos:
                sleep(0.5)
                waitAndPress("enter")
        else:
            result.append({"Personnr": personnr, "Orsak": orsak, "Date": date})
            waitAndPress("alt", "s")
    else:
        pos = waitAndClickElement(IMG.Avbryt)

    waitForElement(IMG.WaitingListMenuBar)
    sleep(0.5)

    if current_personnr_limit_first_part in personnr and current_personnr_limit_last_part in personnr: # ADD THE VALID PERSONNR 
        break
    waitAndPress("down")
    sleep(0.4)
    if final_personnr_limit_first_part in personnr and final_personnr_limit_last_part in personnr:
        waitAndPress("down")
        sleep(0.4)

    
    
df = pd.DataFrame(result)
df.to_excel("JK_waitingListWithComplaint.xlsx", index=False)

In [ ]:
df = pd.DataFrame(result)
df.to_excel(file_path+"JK_waitingListWithComplaint.xlsx", index=False)

# Add diagnoses

In [ ]:
df = pd.read_excel(file_path+"JK_waitingListWithComplaint_2024-11.xlsx")

In [ ]:
def format_personnr(original):
    # Convert to string
    formatted = f"{int(float(original)):012}" if isinstance(original, (float, int)) else f"{original:>012}"
    
    delimiter = "-"

    date_born_str = re.sub(r"(\d\d\d\d)(\d\d)(\d\d)\d\d\d\d", r"\1-\2-\3", formatted)

    date_born = string_to_date(date_born_str)

    original_date = string_to_date("2024-11-20")

    if (original_date - date_born).days > 100 * 365.25:
        delimiter = r"+"

    formatted = re.sub(r"\d\d(\d\d\d\d\d\d)(\d\d\d\d)", r"\1"+delimiter+r"\2", formatted)

    return formatted

format_personnr("192001011234")

In [ ]:
patient_diagnoses_df = pd.read_excel(file_path+"Diagnoser.xlsx")
print(len(patient_diagnoses_df.index))
patient_diagnoses_df = patient_diagnoses_df.loc[~(patient_diagnoses_df["Personnr"].isna()) & (patient_diagnoses_df["Personnr"] != "")].copy()
print(len(patient_diagnoses_df.index))

In [ ]:
patient_diagnoses_df.loc[:, "Personnr"] = patient_diagnoses_df.loc[:, "Personnr"].apply(format_personnr)
patient_diagnoses_df.loc[:, "Diagnos"] = patient_diagnoses_df.loc[:, "Diagnos"].str.replace(r",.+", "")
# patient_diagnoses_df.head(2)

In [ ]:
diagnoses_dict_df = pd.read_excel(file_path+"DiagnosDict2.xlsx")
diagnoses_dict_df.head()

In [ ]:
diagnoses_dict = {}
for index, row in diagnoses_dict_df.iterrows():
    diagnoses_dict[row["icd"]] = row["bokningsnamn"]

# {'I109': 'hypertoni',
#  'R529': 'smärta',
#  'J459': 'astma',
# (...)

In [ ]:
example_row = df.loc[10,:]

def create_diagnosis_list(row):
    personnr = row["Personnr"]
    icd_set = set()
    diagnoses = []

    for index, row in patient_diagnoses_df.iterrows():
        if personnr == row["Personnr"]:
            icd_set.add(row["Diagnos"])

    if len(icd_set) == 0:
        return ""
    
    icd_list = list(icd_set)
    icd_list.sort()

    for icd in icd_list:
        if icd in diagnoses_dict:
            diagnoses.append(diagnoses_dict[icd])

    return ", ".join(diagnoses)


create_diagnosis_list(example_row)

In [ ]:
def create_new_complaint_string(row):
    new = row["Orsak"]
    diagnoses = create_diagnosis_list(row)
    if diagnoses and ("årlig" in new or "ÅRLIG" in new):
        new = re.sub(r"^årlig|^ÅRLIG", "", new)
        new = "årlig " + diagnoses + new
    
    return new

create_new_complaint_string(example_row)

In [ ]:
df["Orsak uppdaterad"] = df.apply(create_new_complaint_string, axis="columns")

In [ ]:
total = len(df.index)
changed = len(df.loc[ df["Orsak"] != df["Orsak uppdaterad"] ].index)
print(f"Changed {changed}/{total}")

In [ ]:
df.to_excel(file_path+"JK_waitinglist_updated_complaint.xlsx", index=False)

# Apply changes

In [ ]:
df = pd.read_excel(file_path+"JK_waitinglist_updated_complaint.xlsx")
df = df.loc[ df["Orsak"] != df["Orsak uppdaterad"] ]

In [ ]:
complaint_dict = {}

for index, row in df.iterrows():
    complaint_dict[row["Personnr"] + "_" + row["Date"] + "_" + row["Orsak"]] = row["Orsak uppdaterad"]

In [ ]:
result = []
OpenPMO()
for i in range(500):
    waitAndPress("alt", "A")
    waitForElement(IMG.Avbryt)
    sleep(0.5)

    alive = check_alive()

    pos = waitAndClickElement(IMG.Orsak, relX=100, relY=100, beforeStart=0.5)
    if pos:
        waitAndPress("ctrl", "A")
        waitAndPress("ctrl", "C")
        sleep(0.6)
        orsak = str(pyperclip.paste())
    else:
        break

    if not "ADM " in orsak.upper():
        pos = waitForElement(IMG.Patienter)
        if pos:
            personnr = capture_and_ocr(pos.x-14, pos.y+12, 100, 20)
            personnr = personnr.replace("—", "-")
        else:
            break

        pos = waitAndClickElement(IMG.BokaSenast, relY=15)
        if pos:
            waitAndPress("ctrl", "A")
            waitAndPress("ctrl", "C")
            sleep(0.4)
            date = str(pyperclip.paste())
        else:
            break

        dict_key = f"{personnr}_{date}_{orsak}"

        if not alive:
            print(f"{date}, {personnr} är avliden")
            pos = waitAndClickElement(IMG.Avbryt)
            if pos:
                sleep(0.5)
                waitAndPress("enter")
        elif dict_key in complaint_dict:
            pos = waitAndClickElement(IMG.Orsak, relX=100, relY=100, beforeStart=0.5)
            if pos:
                waitAndPress("ctrl", "A")
                sleep(0.4)
                waitAndType(complaint_dict[dict_key])
                sleep(0.4)

            print("Updated " + personnr)
            
            waitAndPress("alt", "s")
        else:
            print("Did not update " + personnr)
            pos = waitAndClickOneOf([IMG.Avbryt, IMG.Avbryt2])
    else:
        pos = waitAndClickOneOf([IMG.Avbryt, IMG.Avbryt2])

    waitForElement(IMG.WaitingListMenuBar)
    sleep(0.5)

    if current_personnr_limit_first_part in personnr and current_personnr_limit_last_part in personnr: # ADD THE VALID PERSONNR 
        break
    waitAndPress("down")
    sleep(0.4)
    if final_personnr_limit_first_part in personnr and final_personnr_limit_last_part in personnr:
        waitAndPress("down")
        sleep(0.4)


    


# Change receiving dr

In [ ]:
originDate = string_to_date("2024-12-08")
def format_waitlist_personnr(personnr):
    """
    Formats the person number by determining the correct century.

    Args:
        personnr (str): The person number in the format "YYMMDD-XXXX" or "YYMMDD+XXXX".
        originDate (datetime): The reference date to determine the century.

    Returns:
        str: The formatted person number in the format "YYYYMMDD-XXXX".
    """
    # Extract the year, month, and day from the personnr
    date_part, suffix = personnr.split("-") if "-" in personnr else personnr.split("+")
    year = int(date_part[:2])
    month_day = date_part[2:]

    # Determine the full year
    current_century = originDate.year // 100
    full_year = current_century * 100 + year

    # If there's a '+' or if the year is over a century in the future, adjust the century
    if "+" in personnr or full_year > originDate.year:
        full_year -= 100

    # Format the full date and return
    formatted_personnr = f"{full_year:04d}{month_day}-{suffix}"
    return formatted_personnr

format_waitlist_personnr("870104-1234")

In [ ]:
moves = pd.read_excel(file_path+"all_moves_WITH_EXTRA_ADJUSTMENTS.xlsx")

In [ ]:
initials_to_search_staff_dict = {
    "LW": "linn",
    "MS": "maria m",
    "RA": "ra",
    "AE": "anna k",
    "H": "hana",
    "JK": "juli",
    "MT": "mic",
}

In [ ]:
personnr_to_search_staff_dict = {}

for index, row in moves.iterrows():
    personnr_to_search_staff_dict[row["Personnr"]] = initials_to_search_staff_dict[row["To"]]

In [ ]:
result = []
OpenPMO()
for i in range(97):
    waitAndPress("alt", "A")
    waitForElement(IMG.Avbryt)
    sleep(0.5)

    alive = check_alive()

    if alive:
        pos = waitForElement(IMG.Patienter)
        sleep(0.5)
        if pos:
            personnr = capture_and_ocr(pos.x-14, pos.y+12, 100, 20)
            personnr = personnr.replace("—", "-")
            personnr = format_waitlist_personnr(personnr)
        else:
            break

        if personnr and personnr in personnr_to_search_staff_dict:
            pos = waitAndClickElement(IMG.Kategori, relY=55, relX=100)
            if pos:
                waitAndType(personnr_to_search_staff_dict[personnr])
                waitAndPress("enter")
                print("Updated " + personnr)
                waitAndClickElement(IMG.Orsak, relX=100, relY=100, beforeStart=0.2)
                waitAndType(" ")
                waitAndPress("backspace")
                waitAndPress("alt", "s")
        else:
            print("Did not update " + personnr)
            pos = waitAndClickOneOf([IMG.Avbryt, IMG.Avbryt2])
    else:
        print("Did not update because not alive")
        pos = waitAndClickOneOf([IMG.Avbryt, IMG.Avbryt2])

    waitForElement(IMG.WaitingListMenuBar)
    sleep(0.5)

    if f"{final_personnr_limit_first_part}-{final_personnr_limit_last_part}" in personnr: # ADD THE VALID PERSONNR 
        break
    waitAndPress("down")
    sleep(0.4)
    if f"{current_personnr_limit_first_part}-{current_personnr_limit_last_part}" in personnr:
        waitAndPress("down")
        sleep(0.4)


    